In [1]:
%run PATTERN_OBSERVER.ipynb

In [2]:
from ipywidgets import Text,Output,GridspecLayout # for SINGLE_TAB
from ipywidgets import Tab,Layout # for TAB_COLLECTION

In [3]:
class SINGLE_TAB:
    def __init__(self,
        name='SINGLE_TAB',
        description='SINGLE_TAB_DESCRIPTION',
        cols=3,
        rows=4,
        ):
        self._cols=cols
        self._rows=rows
        self._times_computed=0
        
        self.name=name
        self.description=description
        self.widget=GridspecLayout(n_rows=self._rows,n_columns=self._cols)
        
        # instantiate
        for r in range(self._rows):
            for c in range(self._cols):
                self.widget[r,c]=Output()

        # draw/redraw all plots in grid
        self.update()

    '''update all plots in grid'''
    def update(self):
        self._times_computed+=1
        
        for r in range(self._rows):
            for c in range(self._cols):
                self.widget[r,c].clear_output()
                with self.widget[r,c]:
                    display(Text('row'+str(r)+' vs '+'col'+str(c)+' = '+str(self._times_computed)))
                
SINGLE_TAB().widget

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [4]:
'''
# example
a=SINGLE_TAB()
a.widget
a.update()
'''
None

In [5]:
class TAB_COLLECTION(IObserver):
    def __init__(self,
        name='TAB_COLLECTION',
        description='TAB_COLLECTION_DESCRIPTION',
        tabs=
        [
            SINGLE_TAB('tab1'),
            SINGLE_TAB('tab2'),
            SINGLE_TAB('tab3'),
            SINGLE_TAB('tab4'),
        ],
        ):
        self.tabs=tabs # contents of tab collection
        self.active_tab=0 # root tab selected by default
        self._is_up_to_date=[True]*len(self.tabs) # nothing selected by default, all tabs up to date by default
        
        self.name=name
        self.description=description
        self.widget=self._make()
        
    ####################################
    # observer pattern
    ####################################
    # observer
    def react(self,subject_name,subject_info):
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info)
        
        if subject_name=='DATA_CONTAINER':
            print(1)
            #self._refresh_options(subject_info['all_backtests'])

    ####################################
    # own stuff
    ####################################
    '''change active tab, update as required'''
    def _onchange_tab(self,change):
        print(self.name,':','SELECTED_INDEX',change.new)
        self.active_tab=change.new # change active tab
        
        # handle delayed update
        if not self._is_up_to_date[self.active_tab]:
            print(self.name,':','UPDATE','TAB',self.active_tab)
            self.tabs[self.active_tab].update() # update this particular tab
            self._is_up_to_date[self.active_tab]=True # flag is up to do date
            
        # logging
        self._logging_delayed_updating()
        
    '''build tab collection'''
    def _make(self):
        # make tab collection
        out=Tab(
            children=[x.widget for x in self.tabs],
            layout=Layout(width='100%',height='auto',min_height='500px')
        )

        # name tabs
        for i,tab_name in enumerate([x.name for x in self.tabs]):
            out.set_title(index=i,title=tab_name)
            
        # observe active tab
        out.observe(self._onchange_tab,names='selected_index')

        # set default
        #out.selected_index=2
        #out.selected_index=3

        # return it
        return out

    def _logging(self):
        # delayed updating
        print(self.name,':','TIMES_COMPUTED',self._times_computed)
        print(self.name,':','IS_UP_TO_DATE',self._is_up_to_date)

    '''delayed update of tab collection, update active tab immediate, delay update of non-active tabs till when they are viewed'''
    def update(self):
        # delayed update
        for i,x in enumerate(self.tabs):
            if self.active_tab==i:
                print(self.name,':','UPDATE','TAB',self.active_tab)
                x.update() # update this particular tab
                self._is_up_to_date[i]=True # flag is up to do date
                
            else:
                self._is_up_to_date[i]=False # flag these tabs as out of date, update when viewed
            
        # logging
        self._logging()
        
    @property
    def _times_computed(self):
        return [x._times_computed for x in self.tabs]

# example
TAB_COLLECTION().widget

In [6]:
'''
# example
a=TAB_COLLECTION(tabs=
    [
        SINGLE_TAB('a'),
        SINGLE_TAB('b'),
        SINGLE_TAB('c'),
        SINGLE_TAB('d'),
    ]
)
a.widget

# global delayed update
a.update()

# request immediate update of specific tab
a.tabs[0].update()
a.tabs[0].update()
a.tabs[2].update()
a.tabs[2].update()
a.tabs[3].update()
'''
None